In [4]:
#Loading a dataset for fine-tuning
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [5]:
#load_dataset() : loads a dataset from Hugging Face hub
#imdb: review classification
from datasets import load_dataset

In [9]:
#splitting Data into Train and Test
train_data = load_dataset("imdb", split="train")
train_data = train_data.shard(num_shards=4, index=0)
test_data = load_dataset("imdb", split="test")
test_data = test_data.shard(num_shards=4, index=0)

In [6]:
# Auto Classes
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
# Tokenize the data and convert to dictionary format
import torch

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64, return_tensors="pt")
#padding="max_length" and truncation=True to ensure all sequences have the same length.
tokenized_training_data = train_data.map(preprocess_function, batched=True)
tokenized_training_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_test_data = test_data.map(preprocess_function, batched=True)
tokenized_test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

#set_format converts the tokenized datasets into the correct format expected by the Trainer"""columns=['input_ids', 'attention_mask', 'label']"""


Map:   0%|          | 0/6250 [00:00<?, ? examples/s]

Map:   0%|          | 0/6250 [00:00<?, ? examples/s]

In [18]:
 print(tokenized_training_data)


Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6250
})


In [21]:
# Fine-tuning through training
# Training Arguments customize training settings

from transformers import Trainer,TrainingArguments

training_args = TrainingArguments(
    output_dir="./results", # output directory
    evaluation_strategy="epoch", # when to evaluate "epoch","steps", or "none
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,# per_device_train_batch_size and per_device_eval_batch_size define the batch size
    num_train_epochs=3,
    weight_decay=0.01 # to avoid overfitting
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=tokenized_training_data,
eval_dataset=tokenized_test_data,
tokenizer=tokenizer
)
trainer.train()

<ipython-input-22-a4786d42167a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.000002
2,0.000000,0.000001
3,0.000000,0.000001


TrainOutput(global_step=1173, training_loss=7.734059708773334e-06, metrics={'train_runtime': 335.1511, 'train_samples_per_second': 55.945, 'train_steps_per_second': 3.5, 'total_flos': 616666536000000.0, 'train_loss': 7.734059708773334e-06, 'epoch': 3.0})

In [24]:
 new_data = ["This is movie was disappointing!", "This is the best movie ever!"]

new_input = tokenizer(new_data, return_tensors="pt", padding=True, truncation=True, max_length=64)


with torch.no_grad():
    outputs = model(**new_input)

predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()


In [25]:
label_map = {0: "NEGATIVE", 1: "POSITIVE"}
for i, predicted_label in enumerate(predicted_labels):
  sentiment = label_map[predicted_label]
  print(f"\nInput Text {i + 1}: {new_data[i]}")
  print(f"Predicted Label: {sentiment}")



Input Text 1: This is movie was disappointing!
Predicted Label: NEGATIVE

Input Text 2: This is the best movie ever!
Predicted Label: NEGATIVE
